# JSON Conversions

This notebook demonstrates how to convert a Resource to JSON and vice-versa. 

## Initialisation

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/getting-started"]))

print("The working directory is now:")
!pwd

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

# Imports

In [3]:
import json

In [4]:
from kgforge.core import Resource

In [5]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [6]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [7]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [8]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [9]:
persons = [jane, john]

In [10]:
forge.register(persons)

<count> 2
<action> _register_one
<succeeded> True


In [11]:
association = Resource(type="Association", agent=persons)

In [12]:
forge.register(association)

<action> _register_one
<succeeded> True


In [13]:
pp(forge.as_json(association))

{
    "id": "e5ed50d2-bc03-45c5-9953-4efed4ddfb0d",
    "type": "Association",
    "agent": [
        {
            "id": "983b316e-db52-4308-8c7b-979654cab3c8",
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva"
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "id": "8dd74b04-b9b5-4ad2-a9e0-63251e514ebe",
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}


In [14]:
pp(forge.as_json(jane, store_metadata=True))

{
    "id": "983b316e-db52-4308-8c7b-979654cab3c8",
    "type": "Person",
    "address": {
        "type": "PostalAddress",
        "country": "Switzerland",
        "locality": "Geneva"
    },
    "email": "(missing)",
    "name": "Jane Doe",
    "deprecated": false,
    "version": 1
}


## JSON to Resource

In [15]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [16]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [17]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [18]:
persons = [jane, john]

In [19]:
association = Resource(type="Association", agent=[jane, john])

In [20]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [21]:
resource = forge.from_json(data)

In [22]:
resource == association

True

In [23]:
print(forge.from_json(data, na="(missing)"))

{
    type: Association
    agent:
    [
        {
            type: Person
            address:
            {
                type: PostalAddress
                country: Switzerland
                locality: Geneva
            }
            name: Jane Doe
        }
        {
            type: Person
            email: john.smith@epfl.ch
            name: John Smith
        }
    ]
}
